# Reading Data from an Arduino

In this tutorial, we will read and graph analog data from an Arduino. The package ``pyserial`` is used for serial communication with the Arduino.

I have chosen to write the Arduino program such that it waits and listens to the serial port. When the Python program sends a character to the Arduino, the Arduino responds with a single data point.

## Apparatus

A joystick is connected to an arduino, with the up-down (UD) potentiometer of the Arduino connected to the A0 input of the Arduino and the left-right (LR) potentiometer of the Arduino connect to the A1 input of the Arduino. (The picture below also shows a push-button switch, but it's not used in the Arduino program.)

<img src="06-files/arduino.jpg" width=200>

## Arduino Program

Here is the program running on the Arduino. It listens for a character (from Python) and responds with the joystick voltage values for LR and UD.

```arduino
#define UD A0
#define LR A1
boolean done = false;
int N; // number of measurements to make but one measurement is always returned in this program
int LRvali; // initial LR value
int UDvali; // initial UD value
int LRval; //  LR value
int UDval; //  UD value



void setup() {
  pinMode(13, OUTPUT);
  Serial.begin(9600);
  digitalWrite(13, LOW);   // turn the LED off
  LRvali = analogRead(LR);
  UDvali = analogRead(UD);
}

void loop() {
    	done = false;
    
    	// Check input on serial line.
    	while (!done) {
        //listen to Serial input
    		if (Serial.available()) {	// Something is in the buffer
    			N = Serial.read();	// so get the number byte
    			done = true;
    		}
    	}

      // get joystick values  
      LRval = analogRead(LR);
      UDval = analogRead(UD);

      // write joystick values
      Serial.print(LRval, DEC);
      Serial.print('\t');
      Serial.print(UDval, DEC);
      Serial.print('\n');
}
```

## Set Up Plotly for Streaming Data

In this example I will stream data to plot.ly in realtime. See a previous tutorial on setting up Plotly for streaming.

In [1]:
#for graphing
import plotly.plotly as py  
import plotly.tools as tls   
import plotly.graph_objs as go

#for serial communication and time
import serial, time

In [2]:
# Get stream id from stream id list 
stream_ids = tls.get_credentials_file()['stream_ids']
stream_id = stream_ids[0]

# Make instance of stream id object 
stream_1 = go.Stream(
    token=stream_id  # link stream id to 'token' key
)

# Initialize trace of streaming plot by embedding the unique stream_id
trace1 = go.Scatter(
    x=[],
    y=[],
    mode='lines+markers',
    stream=stream_1         # (!) embed stream id, 1 per trace
)

data = go.Data([trace1])

# Add title to layout object
layout = go.Layout(title='UD Voltage')

# Make a figure object
fig = go.Figure(data=data, layout=layout)

# Send fig to Plotly, initialize streaming plot, open new tab
py.iplot(fig, filename='Joystick UD Voltage as a Function of Time')


# We will provide the stream link object the same token that's associated with the trace we wish to stream to
s = py.Stream(stream_id)

# We then open a connection
s.open()

## Get Data from the Arduino

The purpose of this function is to get data from the Arduino. When called, it writes a byte to the Arduino, and then reads the response from the Arduino. In this case, the Arduino is writes a single tab-delimited measurement for the LR analog voltage and UD analog voltage.

The function returns a list with the single data point for LR and UD voltage.

In [3]:
def getData(s):
    ser.write(bytes(1))
    line = ser.readline()
    data = line.split()
    LR = int(data[0])
    UD = int(data[1])
    return [LR,UD]

## Initialize Serial Communication

Set the baud rate and port. It is easiest to get the port from the Arduino software you used when you downloaded the Arduino program. For faster data retrieval, you will have to increase the baud rate.

In [4]:
baud=9600 #baud rate (bits per second.)
TO=5 #timeout in sec

#arduino 1
port = '/dev/cu.usbmodem1411'
ser = serial.Serial(port, baud, timeout=TO)

#this delay seems to help
time.sleep(2)     #2ms delay

## Read and Graph Data

In a loop, call the `getData()` function and plot the results in Plotly.

In [5]:
start=time.time()
while True:
    results=getData(ser)  #gets result from arduino[1] 

    # Current time on x-axis, random numbers on y-axis
    x = time.time()-start
    y = results[1]
    
#    print(x,y)

    # Send data to your plot
    s.write(dict(x=x, y=y))

    #     Write numbers to stream to append current data on plot,
    #     write lists to overwrite existing data on plot

    time.sleep(0.1)  # sleep in seconds  
    
s.close()

KeyboardInterrupt: 